In [7]:
%load_ext autoreload
%autoreload 2

from state import State
from domain import Domain
from swe_operators import SweAdvectiveFormOperator
from time_schemes import rk4
from initial_conditions import gaussian_hill, barotropic_instability, geostrophic_balance
import operators as op
from vizualization import trisurf_field, animate_trisurf, surf_interpolated_field, animate_interpolated_surf, \
                          animate_tripcolor, tripcolor_field, imshow_interpolated_field

import numpy as np
from vecmath import calc_mass, calc_c_norm
%matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# definition of computational domain constants
RADZ = 6371.22 * 1000.0 # "Earth" radii in meters
GRAV = 9.80616 # "Earth" gravity acceleration in m * s**(-2)
OMEGA = 7.292 * 10**(-5) # "Earth" angular velocity in rad * s**(-1)
PCORI = 2 * OMEGA # Coriolis parameter
H_MEAN = 10 * 1000.0 # "Earth" mean atmosphere depth
C_GRAV = np.sqrt(GRAV * H_MEAN) # Gravity wave speed
LX = 2 * np.pi * RADZ # domain size [0, LX] x [0, LY]
LY = 2 * np.pi * RADZ # domain size [0, LX] x [0, LY]

Несколько примеров использования

# Разбегание горки, два блока с одинаковым разрешением

In [9]:
%%time
import time
from IPython.display import display, clear_output

domains = [Domain(xs = 0.0,    xe = LX / 2, nx = 32, ys = 0.0, ye = LY, ny = 64,  lx=LX, ly=LY), \
           Domain(xs = LX / 2, xe = LX    , nx = 32, ys = 0.0, ye = LY, ny = 64,  lx=LX, ly=LY)]

state = gaussian_hill(domains, H_MEAN)
swe_operator = SweAdvectiveFormOperator(g=GRAV, pcori=0.0, diff_method = op.diff_sbp21)

T = 2 * 24 * 3600
Nt = 2* 120
dt = T / Nt

states_list = [state]
for k in range(Nt+1):
    clear_output(wait=True)
    display('Time step '+ str(k))
    state = rk4(state, swe_operator, dt, domains)
    states_list.append(state)

'Time step 240'

CPU times: user 3.98 s, sys: 787 ms, total: 4.76 s
Wall time: 4.25 s


In [5]:
h_list = [state.h for state in states_list]

In [ ]:
animate_trisurf(h_list, domains, 4)

In [6]:
import matplotlib.pyplot as plt
c_norms = np.empty([])

for i in range(np.size(states_list)):
    c_norms = np.append(c_norms, calc_c_norm(states_list[0].h-states_list[i].h, domain))

NameError: name 'calc_c_norm' is not defined

In [ ]:
plt.plot(c_norms[1:])
plt.grid()